# Regresión logística

Ya sabemos cómo funciona una red neuronal y ya sabemos implementarla mediante Pytorch, así que vamos a empezar a ver el ejemplo más sencillo de una red neuronal, la regresión logística.

Importamos la base de datos de precios de casas de Boston

In [53]:
from sklearn import datasets

boston = datasets.load_boston()

Podemos ver qué trae esta base de datos

In [54]:
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

La llave `DESCR` es una descripción de la base de datos

In [55]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Además tiene las llaves `data` y `target` donde se encuentran los datos anteriormente descritos. La llave `feature_names` contiene los numbres de cada una de las características

Así que creamos un dataframe con los datos

In [56]:
import pandas as pd

boston_df = pd.DataFrame(boston['data'], columns=boston['feature_names'])
boston_df['target'] = boston['target']
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


Por último vamos a ver si hay algún dato faltante

In [57]:
boston_df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
target     0
dtype: int64

## División de los datos en train y validacion

Para poder entrenar hemos visto que necesitamos dividir los datos en un conjunto de datos de entrenamiento y en un conjunto de datos de validación. Así que dividimos nuestros datos en estos dos conjuntos.

Primero vamos a ver cuantos datos tenemos

In [58]:
len(boston_df)

506

Como no tenemos muchos datos vamos a dividir el conjunto de datos en un 80% para entrenamiento entrenamiento y un 20% para validación

In [59]:
boston_train = boston_df.iloc[0:int(0.8*len(boston_df)), :]
boston_val = boston_df.iloc[int(0.8*len(boston_df)):, :]

len(boston_train), len(boston_val), len(boston_df), len(boston_train) + len(boston_val)

(404, 102, 506, 506)

Vemos como quedan los dos nuevos dataframes

In [60]:
boston_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [61]:
boston_val.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
404,41.52920,0.0,18.1,0.0,0.693,5.531,85.4,1.6074,24.0,666.0,20.2,329.46,27.38,8.5
405,67.92080,0.0,18.1,0.0,0.693,5.683,100.0,1.4254,24.0,666.0,20.2,384.97,22.98,5.0
406,20.71620,0.0,18.1,0.0,0.659,4.138,100.0,1.1781,24.0,666.0,20.2,370.22,23.34,11.9
407,11.95110,0.0,18.1,0.0,0.659,5.608,100.0,1.2852,24.0,666.0,20.2,332.09,12.13,27.9
408,7.40389,0.0,18.1,0.0,0.597,5.617,97.9,1.4547,24.0,666.0,20.2,314.64,26.40,17.2


Vemos que al dividir el dataframe, en el dataframe de validación los índices no empiezan en 0. Esto nos puede dar problemas más adelante, así que reseteamos los índices

In [62]:
boston_val = boston_val.reset_index()   # reset index
boston_val = boston_val.drop(columns=['index']) # drop old index

boston_val.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,41.52920,0.0,18.1,0.0,0.693,5.531,85.4,1.6074,24.0,666.0,20.2,329.46,27.38,8.5
1,67.92080,0.0,18.1,0.0,0.693,5.683,100.0,1.4254,24.0,666.0,20.2,384.97,22.98,5.0
2,20.71620,0.0,18.1,0.0,0.659,4.138,100.0,1.1781,24.0,666.0,20.2,370.22,23.34,11.9
3,11.95110,0.0,18.1,0.0,0.659,5.608,100.0,1.2852,24.0,666.0,20.2,332.09,12.13,27.9
4,7.40389,0.0,18.1,0.0,0.597,5.617,97.9,1.4547,24.0,666.0,20.2,314.64,26.40,17.2


## Dataset y Dataloader

Creamos el dataset

In [63]:
import torch

class BostonDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        cols = [col for col in dataframe.columns if col != 'target']
        self.parameters = dataframe[cols].values.astype('float32')
        self.targets = dataframe['target'].values.astype('float32')
        self.targets = self.targets.reshape((len(self.targets), 1))

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        parameters = self.parameters[idx]
        target = self.targets[idx]
        return parameters, target

In [64]:
train_ds = BostonDataset(boston_train)
valid_ds = BostonDataset(boston_val)
len(train_ds), len(valid_ds)

(404, 102)

vemos que tienen el mismo tamaño que los dataframes de entrenamiento y validación

Vamos a ver una muestra

In [65]:
boston_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [66]:
sample = train_ds[0]
print(f"len(sample): {len(sample)}")

parameters, target = sample
print(f"parameters: {parameters}\nparameters.dtype: {parameters.dtype}\nparameters.shape: {parameters.shape}\n\n")
print(f"target: {target}, target.dtype: {target.dtype}, target.shape: {target.shape}")

len(sample): 2
parameters: [6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]
parameters.dtype: float32
parameters.shape: (13,)


target: [24.], target.dtype: float32, target.shape: (1,)


Creamos ahora el dataloader

In [67]:
from torch.utils.data import DataLoader

BS_train = 32
BS_val = 1024

train_dl = DataLoader(train_ds, batch_size=BS_train, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=BS_val, shuffle=False)

Vemos un batch

In [68]:
batch = next(iter(train_dl))
parameters, target = batch[0], batch[1]
type(parameters), parameters.dtype, parameters.shape, type(target), target.shape

(torch.Tensor,
 torch.float32,
 torch.Size([32, 13]),
 torch.Tensor,
 torch.Size([32, 1]))

## Red Neuronal

Creamos una red neuronal para entrenarla

In [69]:
from torch import nn

class BostonNeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_layers=[20, 8, 3]):
        super().__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, hidden_layers[0]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(hidden_layers[0], hidden_layers[1]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(hidden_layers[1], hidden_layers[2]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(hidden_layers[2], num_outputs)
        )

    def forward(self, x):
        logits = self.network(x)
        return logits

Vamos a ver qué tamaño necesitamos a la entrada y a la salida de la red

Un batch tiene unos parámetros con este tamaño

In [70]:
parameters.shape

torch.Size([32, 13])

Tenemos una matriz de tamaño 32x13. 32 es el tamaño del batch size, mientras que 13 es el número de parámetros, por lo que a la entrada necesitamos 13 neuronas

Otra forma de verlo es que como se tiene que hacer una multiplicación matricial de las entradas con la primera capa de la red, si la matriz de entradas tiene un tamaño de 32x13, la matriz que representa las neuronas de la primera capa tiene que tener un tamaño de 13xM. Ya que en una multiplicación matricial, el tamaño de las matrices que se multiplican tienen que ser AxB y BxC, es decir, la dimensión de en medio de las dos matrices tiene que ser la misma

Por otro lado, el mismo batch a la salida tiene un target con este tamaño

In [71]:
target.shape

torch.Size([32, 1])

32 es el tamaño del batch size, pero solo hay 1 target, que es el precio de las casas, por lo que a la salida queremos que haya solo 1 neurona

In [72]:
num_inputs = parameters.shape[1]
num_outputs = target.shape[1]
model = BostonNeuralNetwork(num_inputs, num_outputs)

model

BostonNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=20, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=20, out_features=8, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=8, out_features=3, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=3, out_features=1, bias=True)
  )
)

Primero cogemos un batch del dataloader y se lo metemos a la red a ver si funciona y la hemos definido bien

In [73]:
preds = model(parameters)
preds.shape

torch.Size([32, 1])

Si se puede se manda la red a la GPU

In [74]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model.to(device)

Using cuda device


BostonNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=20, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=20, out_features=8, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=8, out_features=3, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=3, out_features=1, bias=True)
  )
)

Ahora volvemos a probar a meterle un batch

In [75]:
parameters = parameters.to(device)
preds = model(parameters)
preds.shape

torch.Size([32, 1])

## Función de pérdida y optimizador

Definimos una función de pérdida y un optimizador

In [76]:
LR = 1e-3

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)


## Ciclo de entrenamiento

Entrenamos la red

In [77]:
num_prints = 2

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % int(len(dataloader)/(num_prints-1)) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val_loop(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

Entrenamos

In [78]:
epochs = 14
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
    val_loop(val_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 609.050415  [    0/  404]
loss: 605.313599  [  192/  404]
loss: 544.744507  [  240/  404]
Test Error: 
 Avg loss: 246.350342 

Epoch 2
-------------------------------
loss: 674.444458  [    0/  404]
loss: 682.284912  [  192/  404]
loss: 527.113281  [  240/  404]
Test Error: 
 Avg loss: 210.580032 

Epoch 3
-------------------------------
loss: 580.616455  [    0/  404]
loss: 562.147034  [  192/  404]
loss: 435.839447  [  240/  404]
Test Error: 
 Avg loss: 173.310211 

Epoch 4
-------------------------------
loss: 601.872192  [    0/  404]
loss: 565.008423  [  192/  404]
loss: 255.586136  [  240/  404]
Test Error: 
 Avg loss: 135.926834 

Epoch 5
-------------------------------
loss: 441.121643  [    0/  404]
loss: 376.370117  [  192/  404]
loss: 429.994537  [  240/  404]
Test Error: 
 Avg loss: 102.308777 

Epoch 6
-------------------------------
loss: 368.822479  [    0/  404]
loss: 383.202881  [  192/  404]
loss: 148.840256  [  240/  404]